In [1]:
from pathlib import Path
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px

RES_FILE = Path('results.csv')

In [11]:
res_df = pd.read_csv(RES_FILE)

In [3]:
res_df

,name,table,run_name,time
0,gb_filt_sel_0,tpch_lineitem_1g_no_idx,run_0,0.043674
1,gb_filt_sel_0,tpch_lineitem_1g_no_idx,run_1,0.022029
2,gb_filt_sel_0,tpch_lineitem_1g_no_idx,run_2,0.021127
3,gb_filt_sel_0,tpch_lineitem_1g_no_idx,run_3,0.035106
4,gb_filt_sel_0,tpch_lineitem_1g_no_idx,run_4,0.034149
...,...,...,...,...
655,gb_filt_sel_100,tpch_lineitem_10g,run_5,0.025567
656,gb_filt_sel_100,tpch_lineitem_10g,run_6,0.022164
657,gb_filt_sel_100,tpch_lineitem_10g,run_7,0.021417
658,gb_filt_sel_100,tpch_lineitem_10g,run_8,0.020472


In [38]:
# filter res_df by table name
# split df into multiple dfs by table name
sel_df = res_df[res_df['name'].str.contains('sel')].copy()
sel_df['sel'] = (sel_df['name'].str.rsplit('_', n=1).str[-1]).astype(int)
lg = go.Figure()
tablenames = sel_df['table'].unique()
for table in tablenames:
    df = sel_df[sel_df['table'] == table]
    df = df.sort_values(by=['sel', 'run_name'])
    times: pd.DataFrame = df.groupby('sel', as_index=False)['time'].min()
    # only keep multiples of 20
    # times = times.loc[times['sel'] % 20 == 0]
    print(df, times)
    lg.add_trace(go.Line(x=times['sel'], y=times['time'], name=table.removeprefix('tpch_lineitem_')))
    # break
    # # fig = px.line(df, x='size', y='time', title=f'{table} - Time vs Size')
    # # fig.show()
lg.show()

# for each df, plot the results
# save the results


                name                    table run_name      time  sel
0      gb_filt_sel_0  tpch_lineitem_1g_no_idx    run_0  0.043674    0
1      gb_filt_sel_0  tpch_lineitem_1g_no_idx    run_1  0.022029    0
2      gb_filt_sel_0  tpch_lineitem_1g_no_idx    run_2  0.021127    0
3      gb_filt_sel_0  tpch_lineitem_1g_no_idx    run_3  0.035106    0
4      gb_filt_sel_0  tpch_lineitem_1g_no_idx    run_4  0.034149    0
..               ...                      ...      ...       ...  ...
105  gb_filt_sel_100  tpch_lineitem_1g_no_idx    run_5  0.034899  100
106  gb_filt_sel_100  tpch_lineitem_1g_no_idx    run_6  0.022418  100
107  gb_filt_sel_100  tpch_lineitem_1g_no_idx    run_7  0.039221  100
108  gb_filt_sel_100  tpch_lineitem_1g_no_idx    run_8  0.034521  100
109  gb_filt_sel_100  tpch_lineitem_1g_no_idx    run_9  0.049596  100

[110 rows x 5 columns]     sel      time
0     0  0.020512
1    10  0.018896
2    20  0.019125
3    30  0.019369
4    40  0.020235
5    50  0.021051
6    60  0

/home/kroot/.virtualenvs/pinot-index-benchmarking-9NES4Gek-py3.11/lib/python3.11/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


/home/kroot/.virtualenvs/pinot-index-benchmarking-9NES4Gek-py3.11/lib/python3.11/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


/home/kroot/.virtualenvs/pinot-index-benchmarking-9NES4Gek-py3.11/lib/python3.11/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_ob

# StarTree Index sizes

In [40]:
def extract_st_idx_size(file:Path):
    """Takes CSV file of the form:
    Property,Sizes
    0.null.STAR_TREE.SIZE,"1438573, 309865, 2961549"
    0.l_receiptdate.FORWARD_INDEX.SIZE,"193607, 94334, 315228
    ...
    First column is the property name (table column)
    And second column is the size of each segment on that server

    Returns sum of all sizes across all segments
    """
    df = pd.read_csv(file)
    sizes = df['Sizes']
    sizes = sizes.str.split(', ')
    sizes = sizes.apply(lambda x: sum(map(int, x)))
    return sizes.sum()
    

In [41]:
def extract_threshold(name:str):
    """1g_star_idx_10_sizes_server1.csv -> 10"""
    return int(name.split('_')[3])

In [43]:
from collections import defaultdict

In [44]:
STAR_TREE_SIZES_DIR = Path('results/star_tree_index_size_T/')
assert STAR_TREE_SIZES_DIR.exists()

sizes = defaultdict(int)

for server_dir in STAR_TREE_SIZES_DIR.iterdir():
    server_name = server_dir.name
    print(server_name)
    for file in server_dir.iterdir():
        if "index_map" in file.name:
            continue
        threshold = extract_threshold(file.name)
        size = extract_st_idx_size(file)
        sizes[threshold] += size

sizes = dict(sizes)

        

csv_out_star_T_server_2
csv_out_star_T_server_1
